# Modules

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import sys
import gc
import random

import numpy as np
import pandas as pd

import torch
from torch.nn import MSELoss, L1Loss, SmoothL1Loss, CTCLoss
from torch.optim import Adam, RMSprop
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, StrMethodFormatter
import matplotlib
import seaborn as sns

sys.path.append('..')
from src import config, dataset, deep_models

plt.rcParams.update(plt.rcParamsDefault)
plt.style.use(['ggplot', 'bmh'])
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

display_settings = {
    'max_columns': 999,
    'expand_frame_repr': True,
    'max_rows': 999,
    'precision': 4,
    'show_dimensions': True
}

for op, value in display_settings.items():
    pd.set_option("display.{}".format(op), value)


def fix_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)

fix_seed(1234)
%matplotlib inline

# Data

In [3]:
horizon = 28
last_day = 1913
first_day = 1

dtypes_sales = {
    'id': 'str',
    'item_id': 'str',
    'dept_id': 'str',
    'cat_id': 'str',
    'store_id': 'str',
    'state_id': 'str',
}
dtypes_sales_ts = {f'd_{day}': 'int16' for day in range(1, last_day)}
cat_cols_sales = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

df = pd.read_csv(config.DATA_FILES['sales'], dtype=dict(dtypes_sales, **dtypes_sales_ts))
df[cat_cols_sales] = df[cat_cols_sales].astype('category')

# append days to forecast
for i in range (horizon):
    df[f'd_{last_day + i + 1}'] = np.nan

# categories to ordinal
for col in df.select_dtypes('category'):
    df[col] = df[col].cat.codes

# debug
#df = df.iloc[:3]
    
print(f'Sales series dataset:{df.shape}')
df.head()

Sales series dataset:(30490, 1947)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299,d_300,d_301,d_302,d_303,d_304,d_305,d_306,d_307,d_308,d_309,d_310,d_311,d_312,d_313,d_314,d_315,d_316,d_317,d_318,d_319,d_320,d_321,d_322,d_323,d_324,d_325,d_326,d_327,d_328,d_329,d_330,d_331,d_332,d_333,d_334,d_335,d_336,d_337,d_338,d_339,d_340,d_341,d_342,d_343,d_344,d_345,d_346,d_347,d_348,d_349,d_350,d_351,d_352,d_353,d_354,d_355,d_356,d_357,d_358,d_359,d_360,d_361,d_362,d_363,d_364,d_365,d_366,d_367,d_368,d_369,d_370,d_371,d_372,d_373,d_374,d_375,d_376,d_377,d_378,d_379,d_380,d_381,d_382,d_383,d_384,d_385,d_386,d_387,d_388,d_389,d_390,d_391,d_392,d_393,d_394,d_395,d_396,d_397,d_398,d_399,d_400,d_401,d_402,d_403,d_404,d_405,d_406,d_407,d_408,d_409,d_410,d_411,d_412,d_413,d_414,d_415,d_416,d_417,d_418,d_419,d_420,d_421,d_422,d_423,d_424,d_425,d_426,d_427,d_428,d_429,d_430,d_431,d_432,d_433,d_434,d_435,d_436,d_437,d_438,d_439,d_440,d_441,d_442,d_443,d_444,d_445,d_446,d_447,d_448,d_449,d_450,d_451,d_452,d_453,d_454,d_455,d_456,d_457,d_458,d_459,d_460,d_461,d_462,d_463,d_464,d_465,d_466,d_467,d_468,d_469,d_470,d_471,d_472,d_473,d_474,d_475,d_476,d_477,d_478,d_479,d_480,d_481,d_482,d_483,d_484,d_485,d_486,d_487,d_488,d_489,d_490,d_491,d_492,d_493,...,d_1443,d_1444,d_1445,d_1446,d_1447,d_1448,d_1449,d_1450,d_1451,d_1452,d_1453,d_1454,d_1455,d_1456,d_1457,d_1458,d_1459,d_1460,d_1461,d_1462,d_1463,d_1464,d_1465,d_1466,d_1467,d_1468,d_1469,d_1470,d_1471,d_1472,d_1473,d_1474,d_1475,d_1476,d_1477,d_1478,d_1479,d_1480,d_1481,d_1482,d_1483,d_1484,d_1485,d_1486,d_1487,d_1488,d_1489,d_1490,d_1491,d_1492,d_1493,d_1494,d_1495,d_1496,d_1497,d_1498,d_1499,d_1500,d_1501,d_1502,d_1503,d_1504,d_1505,d_1506,d_1507,d_1508,d_1509,d_1510,d_1511,d_1512,d_1513,d_1514,d_1515,d_1516,d_1517,d_1518,d_1519,d_1520,d_1521,d_1522,d_1523,d_1524,d_1525,d_1526,d_1527,d_1528,d_1529,d_1530,d_1531,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541,d_1542,d_1543,d_1544,d_1545,d_1546,d_1547,d_1548,d_1549,d_1550,d_1551,d_1552,d_1553,d_1554,d_1555,d_1556,d_1557,d_1558,d_1559,d_1560,d_1561,d_1562,d_1563,d_1564,d_1565,d_1566,d_1567,d_1568,d_1569,d_1570,d_1571,d_1572,d_1573,d_1574,d_1575,d_1576,d_1577,d_1578,d_1579,d_1580,d_1581,d_1582,d_1583,d_1584,d_1585,d_1586,d_1587,d_1588,d_1589,d_1590,d_1591,d_1592,d_1593,d_1594,d_1595,d_1596,d_1597,d_1598,d_1599,d_

## Create sequences

In [152]:
# parameters
categories = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
H = 28 # look ahead size
L = 3 * H # look back size
seq_train_last_d = 1913
seq_first_day = seq_train_last_d - 6 * (L + H)
seq_test_last_d = seq_train_last_d + H
S = 7 # stride step
N = 1 + (seq_train_last_d - seq_first_day - L - H) // S # number of subsequences per sequences


ids = df.id.values
sequences_train = df[[f'd_{i}' for i in np.arange(seq_first_day, seq_train_last_d + 1, 1)]].values
covariates = df[categories].values
covariates_size = [df[c].nunique() for c in categories]

X_train_seq = np.empty((N*len(ids), L), dtype=np.int32)
X_train_cov = np.empty((N*len(ids), covariates.shape[1]), dtype=np.int32)
y_train = np.empty((N*len(ids), H), dtype=np.int32)

for i in range(len(ids)):
    n = sequences_train[i].strides[0]
    subsequences = np.lib.stride_tricks.as_strided(sequences_train[i], shape=(N, L + H), strides=(S*n, n))
    X_train_seq[i*N:(i+1)*N, :] = subsequences[:, :-H]
    X_train_cov[i*N:(i+1)*N, :] = np.repeat(covariates[i, np.newaxis], N, 0)
    y_train[i*N:(i+1)*N, :] = subsequences[:, -H:]
    
#X_val_seq = sequences_train[:, -(H+L):-(H)]
#X_val_covariates = covariates
#y_val_seq = sequences_train[:, -H:]

embedding_sizes = [(n_categories, min(50, (n_categories+1)//2)) for n_categories in covariates_size]

train_ids, val_inds = train_test_split(np.arange(N*len(ids)), test_size=.1)

X_val_seq = X_train_seq[val_inds]
X_val_cov = X_train_cov[val_inds]
y_val = y_train[val_inds]

X_train_seq = X_train_seq[train_ids]
X_train_cov = X_train_cov[train_ids]
y_train = y_train[train_ids]

X_test_seq = sequences_train[:, -L:]
X_test_cov = covariates

print('Training')
print(f'Sequences: {X_train_seq.shape}')
print(f'Covariates: {X_train_cov.shape}')
print(f'Label: {y_train.shape}')

print('\nValidation')
print(f'Sequences: {X_val_seq.shape}')
print(f'Covariates: {X_val_cov.shape}')
print(f'Label: {y_val.shape}')

print('\nTest')
print(f'Sequences: {X_test_seq.shape}')
print(f'Covariates: {X_test_cov.shape}')

print('\nCategories embedding sizes')
print(embedding_sizes)

Training
Sequences: (2222721, 84)
Covariates: (2222721, 5)
Label: (2222721, 28)

Validation
Sequences: (246969, 84)
Covariates: (246969, 5)
Label: (246969, 28)

Test
Sequences: (30490, 84)
Covariates: (30490, 5)

Categories embedding sizes
[(3049, 50), (7, 4), (3, 2), (10, 5), (3, 2)]


# Model

In [264]:
batch_size = N * 3

ds_train = TensorDataset(torch.tensor(X_train_seq).float(), 
                         torch.tensor(X_train_cov).long(),
                         torch.tensor(y_train).float())
dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)

ds_val = TensorDataset(torch.tensor(X_val_seq).float(),
                       torch.tensor(X_val_cov).long(),
                       torch.tensor(y_val).float())
dl_val = DataLoader(ds_val, batch_size=batch_size, shuffle=True)

ds_test = TensorDataset(torch.tensor(X_test_seq).float(),
                        torch.tensor(X_test_cov).long())
dl_test = DataLoader(ds_test, batch_size=batch_size, shuffle=False)

print(f'Batch size: {batch_size}')

Batch size: 243


In [194]:
print('Train model...')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

n_epochs = 100
learning_rate=0.001
patience, trials = 100, 0
verbose = 1
loss_history = []
eval_loss_history = []
best_eval_loss = 0

model = deep_models.ARCNN(ar_input_size=L, ar_size_hidden=L, ar_n_hidden=1,
                          cnn_n_kernels_global=L, cnn_n_kernels_local=L//7, cnn_kernel_size_local=7, cnn_dropout=.1,
                          cov_embedding_sizes=embedding_sizes, cov_embedding_dropout=.01, 
                          linear_sizes=[224, 112, 56], linear_dropout=.1, output_size=H).to(device)

#print(model)

criterion = MSELoss(reduction='sum')
optimizer = Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, n_epochs + 1):
    
    # perform a full pass on the dataset
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(dl_train):
        X_seq, X_cov, y = [ds.to(device) for ds in batch]
        
        # erase gradients and perform a forward pass
        optimizer.zero_grad()
        y_hat = model(X_seq, X_cov)
        
        # compute loss and perform a backward pass
        loss = criterion(y_hat, y)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    epoch_loss /= len(ds_train)
    loss_history.append(epoch_loss)
    
    # evaluate
    model.eval()
    eval_loss = 0
    
    for batch in dl_val:
        X_seq, X_cov, y = [ds.to(device) for ds in batch]
        y_hat = model(X_seq, X_cov)
        loss = criterion(y_hat, y)
        eval_loss += loss.item()
    
    eval_loss /= len(ds_val)
    eval_loss_history.append(eval_loss)

    if epoch % verbose == 0:
        print(f'Epoch: {epoch:3d} - Train loss: {epoch_loss:.4f} | Eval loss: {eval_loss:.4f}')

    if eval_loss > best_eval_loss:
        trials = 0
        best_eval_loss = eval_loss
        #torch.save(model.state_dict(), 'best.pth')
        #print(f'Epoch {epoch} best loss: {best_eval_loss:4f}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Train model...
Epoch:   1 - Train loss: 145.0179 | Eval loss: 145.2332
Epoch:   2 - Train loss: 140.6602 | Eval loss: 141.1524
Epoch:   3 - Train loss: 139.2386 | Eval loss: 141.0192
Epoch:   4 - Train loss: 138.4564 | Eval loss: 141.6735
Epoch:   5 - Train loss: 138.0892 | Eval loss: 139.3578
Epoch:   6 - Train loss: 137.6346 | Eval loss: 140.7513
Epoch:   7 - Train loss: 137.3722 | Eval loss: 138.9194
Epoch:   8 - Train loss: 137.0878 | Eval loss: 139.2352
Epoch:   9 - Train loss: 136.8611 | Eval loss: 137.6148
Epoch:  10 - Train loss: 136.6237 | Eval loss: 138.9338
Epoch:  11 - Train loss: 136.3947 | Eval loss: 136.9930
Epoch:  12 - Train loss: 136.1245 | Eval loss: 138.4514
Epoch:  13 - Train loss: 135.8808 | Eval loss: 137.8265
Epoch:  14 - Train loss: 135.6301 | Eval loss: 138.9885
Epoch:  15 - Train loss: 135.6038 | Eval loss: 136.9605
Epoch:  16 - Train loss: 135.2828 | Eval loss: 137.2238
Epoch:  17 - Train loss: 135.4285 | Eval loss: 137.8460
Epoch:  18 - Train loss: 135.1244

In [ ]:
#torch.save(model.state_dict(), 'best.pth')

# Valdiation set

In [ ]:
y_val = []
y_hat_val = []

model.eval()
for batch in dl_val:
    X_seq, X_cov, y = [t.to(device) for t in batch]
    y_hat = model(X_seq, X_cov)
    
    y_val.extend(y.cpu().data.numpy())
    y_hat_val.extend(y_hat.cpu().data.numpy())

In [ ]:
sample = np.random.choice([i for i in range(len(y_val))])

f, ax = plt.subplots(figsize=(10, 5))
ax.plot(y_val[sample], label='True')
ax.plot(y_hat_val[sample], label='Pred')
plt.legend()

# Predictions

In [263]:
y_hat_test = []

model.eval()
for batch in dl_test:
    X_seq, X_cov = [t.to(device) for t in batch]
    y_hat = model(X_seq, X_cov)
    
    y_hat_test.extend(y_hat.cpu().data.numpy())

df_predictions = pd.DataFrame(y_hat_test)
df_predictions.columns = [f'F{i + 1}' for i, c in enumerate(df_predictions.columns)]
df_predictions = df_predictions.set_index(ids)

df_predictions.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
HOBBIES_1_001_CA_1_validation,1.1523,0.9602,0.8830,0.9424,0.8321,0.8975,1.0530,1.0767,1.0060,0.8736,0.8298,0.8712,0.8644,0.9786,1.0264,0.8602,0.8068,0.8189,0.8169,0.8578,0.9942,0.9415,0.7823,0.7181,0.8020,0.7961,0.8422,0.9854
HOBBIES_1_002_CA_1_validation,0.1623,0.1383,0.0957,0.0827,0.0873,0.1981,0.0759,0.1632,0.1749,0.2144,0.1338,0.2212,0.1099,0.1551,0.2042,0.1711,0.1929,0.2318,0.1737,0.1776,0.2311,0.2047,0.0941,0.0896,0.1761,0.1686,0.2557,0.2810
HOBBIES_1_003_CA_1_validation,0.8664,0.7030,0.6172,0.5737,0.5875,0.7066,0.7584,0.7671,0.6135,0.6383,0.6209,0.6170,0.6755,0.7189,0.6573,0.5929,0.5973,0.5640,0.5361,0.5862,0.7307,0.6384,0.5103,0.4904,0.5220,0.4978,0.5850,0.6324
HOBBIES_1_004_CA_1_validation,2.9168,2.2691,2.0137,2.0460,1.8342,2.0290,2.8385,2.9710,2.2502,1.8435,2.0022,1.7587,2.1775,2.7736,2.7749,2.0468,1.7809,1.6748,1.5082,1.9249,2.6970,2.7773,2.0679,1.7628,1.7386,1.7302,1.8487,2.7294
HOBBIES_1_005_CA_1_validation,2.0433,1.5262,1.4299,1.5339,1.3610,1.3750,1.7732,1.6747,1.4092,1.2300,1.2017,1.2393,1.2974,1.6145,1.5343,1.2408,1.1433,1.2046,1.1865,1.3560,1.7018,1.4450,1.1364,1.0759,1.2361,1.1932,1.2954,1.6203
